In [1]:
import geopandas as gpd
import urban_connector as uc

# Calculate accesibility for AOI

## Area of interest (AOI)

In [2]:
# load AOI polygon
gdf_aoi = gpd.read_file("./ex_data/AOI_01.geojson")
gdf_aoi_boundary = gdf_aoi.loc[2, "geometry"]

## Graph

In [3]:
# Create graph
G = uc.osmnx_graph_from_polygon(gdf_aoi_boundary, 'walk', 4.5)

# Create network nodes and lines
gdf_nodes, gdf_edges, dict_mapping_nodes = uc.create_geodata_from_graph(G)

# Export nodes and lines
gdf_nodes.to_file('./ex_data/nodes_all.geojson', driver='GeoJSON')
gdf_edges[['osmid', 'reversed', 'node_id_st', 'node_id_en', 'geometry']].to_file('./ex_data/edges_all.geojson', driver='GeoJSON')

## Sources

In [4]:
# Set amenities to download and the boundary polygon
amenities = ["hospital"] # "pharmacy", "hospital", "school"

# Download amenities from OSM
gdf_amenities = uc.get_amenities(gdf_aoi_boundary, amenities)
gdf_amenities.to_file(f"./ex_data/amenities_hospital.geojson", driver='GeoJSON')
print(len(gdf_amenities))

# Find all nodes closest to sources
list_node_sources = uc.closest_nodes_to_sources(G, gdf_amenities)

d:\GIT\urban_connector\urban_connector\manager\sources_manager.py:30: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf_amenities = ox.geometries_from_polygon(poly_boundary, tags={"amenity": amenities}).loc[:, ["name", "amenity", "geometry"]].reset_index(drop=True)
d:\GIT\urban_connector\urban_connector\manager\sources_manager.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_amenities.geometry = gdf_amenities.geometry.centroid


20


In [5]:
# Calculate the weighted nodes and edges
gdf_nodes_w, gdf_edges_w = uc.calculate_all_nodes_and_edges(G, gdf_nodes, gdf_edges, 'weight', list_node_sources)

# Export nodes and lines
gdf_nodes_w.to_file('./ex_data/nodes_all_w.geojson', driver='GeoJSON')
gdf_edges_w[['osmid', 'reversed', 'node_id_st', 'node_id_en', 'w_start', 'w_end', 'w_mean', 'geometry']].to_file('./ex_data/edges_all_w.geojson', driver='GeoJSON')

## Isochrones

In [6]:
# Set time limits
list_times = [5, 10, 15]

In [7]:
# Create isochrones, rings and edges with travel times
# long function - output: edges, isochrones, rings
list_costs_buffers = []
for v_cost_limit in list_times:
    df_edges_cost_limit = uc.calculate_service_edges(G, gdf_nodes, gdf_edges, 'weight', v_cost_limit, list_node_sources)
    df_edges_cost_limit = df_edges_cost_limit[['node_id_st', 'node_id_en', 'weight', 'w_start', 'w_end', 'w_mean', 'w_limit', 'geometry']]
    df_edges_cost_limit.to_file(f"./ex_data/edges_weight_{v_cost_limit}.geojson", driver='GeoJSON')
    gdf_buffer = uc.create_isochrones(df_edges_cost_limit, "3857", 5, 100)
    gdf_buffer.to_file(f"./ex_data/isochrones_weight_{v_cost_limit}.geojson", driver='GeoJSON')
    list_costs_buffers.append(gdf_buffer)

gdf_service_areas = uc.merge_isochrones_polygons( list_costs_buffers, 'rings')
gdf_service_areas.to_file("./ex_data/isochrones_rings.geojson", driver='GeoJSON')

# Analyze smaller AOIs

In [8]:
gdf_aoi_small = gdf_aoi.loc[:1]

gdf_aoi_small = gdf_aoi_small.to_crs("3857")
gdf_aoi_small['area_has'] = round(gdf_aoi_small.area/10000, 1)  
gdf_aoi_small = gdf_aoi_small.to_crs("4326")

gdf_aoi_small = gdf_aoi_small[['Name', 'area_has', 'geometry']]
gdf_aoi_small

,Name,area_has,geometry
0,POMPEYA,500.7,"POLYGON Z ((-58.41137 -34.63788 0.00000, -58.4..."
1,MATADEROS,502.6,"POLYGON Z ((-58.51787 -34.66203 0.00000, -58.5..."


In [9]:
gdf_aoi_small_inter = gpd.overlay(gdf_aoi_small, gdf_service_areas, how='intersection')
gdf_aoi_small_inter = gdf_aoi_small_inter.to_crs("3857")
gdf_aoi_small_inter['t_area_m'] =  round(gdf_aoi_small_inter.area/10000, 1) 
gdf_aoi_small_inter.w_limit = "t_"+gdf_aoi_small_inter.w_limit.astype(str)

df_isochrones_data = gdf_aoi_small_inter.pivot_table(index='Name', columns='w_limit', values='t_area_m').reset_index()
df_isochrones_data.fillna(0, inplace=True)
df_isochrones_data

w_limit,Name,t_10,t_15,t_5
0,MATADEROS,0.0,0.3,0.0
1,POMPEYA,115.1,122.0,27.0


In [10]:
gdf_aoi_small = gdf_aoi_small.merge(df_isochrones_data, on='Name', how='left')
gdf_aoi_small

,Name,area_has,geometry,t_10,t_15,t_5
0,POMPEYA,500.7,"POLYGON Z ((-58.41137 -34.63788 0.00000, -58.4...",115.1,122.0,27.0
1,MATADEROS,502.6,"POLYGON Z ((-58.51787 -34.66203 0.00000, -58.5...",0.0,0.3,0.0


In [11]:
gdf_aoi_small['area_t_has'] = round((gdf_aoi_small.t_10 + gdf_aoi_small.t_15 + gdf_aoi_small.t_5), 1)
gdf_aoi_small['area_t_per'] = round(100* gdf_aoi_small['area_t_has'] / (gdf_aoi_small['area_has']), 1)
gdf_aoi_small

,Name,area_has,geometry,t_10,t_15,t_5,area_t_has,area_t_per
0,POMPEYA,500.7,"POLYGON Z ((-58.41137 -34.63788 0.00000, -58.4...",115.1,122.0,27.0,264.1,52.7
1,MATADEROS,502.6,"POLYGON Z ((-58.51787 -34.66203 0.00000, -58.5...",0.0,0.3,0.0,0.3,0.1


## Add a location - POMPEYA

In [12]:
# Create a subgraph to Pompeya, without coverage (15m away)
gdf_aoi_pompeya = gdf_aoi.loc[[0]]

gdf_edges_inter = gpd.overlay(gdf_edges, gdf_aoi_pompeya, how='intersection')
gdf_edges_inter = gpd.overlay(gdf_edges_inter, gdf_service_areas, how='difference')

gdf_edges_inter = gdf_edges_inter.rename_axis('id').reset_index()
gdf_edges_inter = gdf_edges_inter.explode(index_parts=False)

node_list_POMPEYA = list(gdf_edges_inter.node_id_st) + list(gdf_edges_inter.node_id_en)
H = G.subgraph(node_list_POMPEYA)

In [13]:
# calculate edges betweenness centrality
v_k_value = int(len(H.nodes)*0.9)
gdf_edges_centrality = uc.calcualte_edges_centrality(H, gdf_edges_inter, centrality_type='betweenness', nodes_ids=False, v_k=v_k_value, v_weight='weight')
gdf_edges_centrality[['osmid', 'reversed', 'node_id_st', 'node_id_en', 'centrality', 'geometry']].to_file("./ex_data/edges_centrality_betweenness.geojson", driver='GeoJSON')

Calculate betweenness centrality


In [14]:
# calculate nodes closeness centrality
gdf_nodes_centrality = uc.calcualte_edges_centrality(H, gdf_nodes, centrality_type='closeness')
gdf_nodes_centrality.loc[gdf_nodes_centrality.centrality.notnull()].to_file('./ex_data/nodes_centrality_closeness.geojson', driver='GeoJSON')


Calculate closeness centrality


## Recalulate isochrones with new amenity

In [15]:
# Add a new source
gdf_new_sources = gpd.read_file("./ex_data/new_source.geojson")
gdf_new_sources = uc.build_source_gdf(gdf_new_sources, field_category='categoria', field_name="nombre")
gdf_new_sources, list_node_sources_new = uc.add_amenities(gdf_new_sources, gdf_amenities, list_node_sources, G, str_method_near_nodes=['closest'])
gdf_new_sources

,src_id,category,name,geometry
0,src_1,hospital,Hospital Neruropsiquiátrico Braulio A. Moyano,POINT (-58.38384 -34.64078)
1,src_2,hospital,Hospital Borda,POINT (-58.38400 -34.63685)
2,src_3,hospital,Hospital Muñiz,POINT (-58.39254 -34.63934)
3,src_4,hospital,Instituto Doctor C. G. Malbrán,POINT (-58.39060 -34.64188)
4,src_5,hospital,Hospital General De Agudos P. Pińero,POINT (-58.45452 -34.64478)
5,src_6,hospital,Maternidad Ramón Sardá,POINT (-58.40312 -34.63465)
6,src_7,hospital,"Hospital de Pediatría ""Profesor Doctor Juan Pe...",POINT (-58.39371 -34.63109)
7,src_8,hospital,Hospital Complejo Médico de la Policía Federal...,POINT (-58.41061 -34.64077)
8,src_9,hospital,Hospital General de Agudos Jose A. Penna,POINT (-58.41031 -34.64345)
9,src_10,hospital,Hospital General de Agudos Doctor Cosme Argerich,POINT (-58.36557 -34.62811)


In [16]:
# Set time limits
list_times = [5, 10, 15]

# Create isochrones, rings and edges with travel times
# long function - output: edges, isochrones, rings
list_costs_buffers = []
for v_cost_limit in list_times:
    df_edges_cost_limit = uc.calculate_service_edges(G, gdf_nodes, gdf_edges, 'weight', v_cost_limit, list_node_sources_new)
    df_edges_cost_limit = df_edges_cost_limit[['node_id_st', 'node_id_en', 'weight', 'w_start', 'w_end', 'w_mean', 'w_limit', 'geometry']]
    df_edges_cost_limit.to_file(f"./ex_data/new_edges_weight_{v_cost_limit}.geojson", driver='GeoJSON')
    gdf_buffer = uc.create_isochrones(df_edges_cost_limit, "3857", 5, 100)
    gdf_buffer.to_file(f"./ex_data/new_isochrones_weight_{v_cost_limit}.geojson", driver='GeoJSON')
    list_costs_buffers.append(gdf_buffer)

gdf_service_areas = uc.merge_isochrones_polygons( list_costs_buffers, 'rings')
gdf_service_areas.to_file("./ex_data/new_isochrones_rings.geojson", driver='GeoJSON')

In [17]:
gdf_aoi_small_inter_2 = gpd.overlay(gdf_aoi_small, gdf_service_areas, how='intersection')
gdf_aoi_small_inter_2 = gdf_aoi_small_inter_2.to_crs("3857")
gdf_aoi_small_inter_2['t_area_m'] =  round(gdf_aoi_small_inter_2.area/10000, 1) 
gdf_aoi_small_inter_2.w_limit = "t2_"+gdf_aoi_small_inter_2.w_limit.astype(str)

df_isochrones_data_2 = gdf_aoi_small_inter_2.pivot_table(index='Name', columns='w_limit', values='t_area_m').reset_index()
df_isochrones_data_2.fillna(0, inplace=True)
df_isochrones_data_2

w_limit,Name,t2_10,t2_15,t2_5
0,MATADEROS,0.0,0.3,0.0
1,POMPEYA,201.9,153.8,52.6


In [18]:
gdf_aoi_small = gdf_aoi_small.merge(df_isochrones_data_2, on='Name', how='left')
gdf_aoi_small

,Name,area_has,geometry,t_10,t_15,t_5,area_t_has,area_t_per,t2_10,t2_15,t2_5
0,POMPEYA,500.7,"POLYGON Z ((-58.41137 -34.63788 0.00000, -58.4...",115.1,122.0,27.0,264.1,52.7,201.9,153.8,52.6
1,MATADEROS,502.6,"POLYGON Z ((-58.51787 -34.66203 0.00000, -58.5...",0.0,0.3,0.0,0.3,0.1,0.0,0.3,0.0


In [19]:
gdf_aoi_small['area_t2_has'] = round((gdf_aoi_small.t2_10 + gdf_aoi_small.t2_15 + gdf_aoi_small.t2_5), 1)
gdf_aoi_small['area_t2_per'] = round(100* gdf_aoi_small['area_t2_has'] / (gdf_aoi_small['area_has']), 1)
gdf_aoi_small

,Name,area_has,geometry,t_10,t_15,t_5,area_t_has,area_t_per,t2_10,t2_15,t2_5,area_t2_has,area_t2_per
0,POMPEYA,500.7,"POLYGON Z ((-58.41137 -34.63788 0.00000, -58.4...",115.1,122.0,27.0,264.1,52.7,201.9,153.8,52.6,408.3,81.5
1,MATADEROS,502.6,"POLYGON Z ((-58.51787 -34.66203 0.00000, -58.5...",0.0,0.3,0.0,0.3,0.1,0.0,0.3,0.0,0.3,0.1


In [20]:
gdf_aoi_small[['area_t_has', 'area_t2_has', 'area_t_per', 'area_t2_per']]

,area_t_has,area_t2_has,area_t_per,area_t2_per
0,264.1,408.3,52.7,81.5
1,0.3,0.3,0.1,0.1
